# Azure AI Agent service - Bing integration

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [ ]:
#%pip install azure-ai-projects

In [ ]:
import datetime
import os
import sys

from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingGroundingTool, MessageRole, ListSortOrder
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

## 1. Grounding Bing Search

### Connect to AI Foundry

In [ ]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

### Create connection to bing grounding service

In [ ]:
# [START create_agent_with_bing_grounding_tool]
conn_id = os.getenv("AZURE_BING_CONNECTION_ID")

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

### Create agent with Bing Grounding tool

In [ ]:
# Create the agent with the Bing grounding tool
instructions = "You are an AI Agent that can do some web search using Bing"
agent = project_client.create_agent(
        model=os.getenv("MODEL_DEPLOYMENT_NAME"),
        name="my-bing-grounding-agent",
        instructions=instructions,
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )
    # [END create_agent_with_bing_grounding_tool]

print(f"Created agent, ID: {agent.id}")

bing
bing.definitions


### Create THREAD

In [ ]:
# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread, ID: {thread.id}")

### Example #1

In [ ]:
query = "What is the current price of MICROSOFT stock?"

### Create first message in the thread

In [ ]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

### Create first RUN

In [ ]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

### Check status of the RUN and tool calls

In [ ]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage


### Get response

In [ ]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

### Example #2

In [ ]:
query = "find out about the latest iPhone"

In [ ]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

In [ ]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

In [ ]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage

In [17]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: The latest iPhone lineup for 2025 is the iPhone 17 series, which will launch in the fall. Notable models include the iPhone 17, iPhone 17 Pro, and the all-new iPhone 17 Air—Apple’s thinnest iPhone ever, focusing on a sleek design while still packing the latest A19 chip and Apple’s high-refresh-rate display. There’s also the iPhone SE 4, a more budget-friendly model released earlier in the year, featuring a modern edge-to-edge OLED display, the A18 chip, a 48MP camera, and USB-C connectivity. All new iPhones benefit from big improvements in AI (Apple Intelligence), camera technology (including periscope zoom on Pro models), and battery life.

If you’re looking for the best blend of value and features, the standard iPhone 16 is still widely recommended, but the iPhone 17 lineup will set the bar for 2025, especially with major camera and AI upgrades. For those prioritizing design, the iPhone 17 Air is the most stylish and unique option this year【7:1†AppleMagazine】【7:2†CNET

### Example #3

In [18]:
query = "What is the weather informations for Mexico city? Show the report and use emojis and be funny"

In [19]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_biH7wU75tKE0M91eU32Xaw20


In [20]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [21]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage

Step step_9y95BT6yfWTO4yXXwVqMY9Wf status: completed
Step step_SonuY0yeACa9fUIMA8GVWhq4 status: completed
  Tool calls:
    Tool Call ID: call_9TJbnaCjp1MTjMFBIPkcg6Nx
    Type: bing_grounding
    Bing Grounding ID: https://api.bing.microsoft.com/v7.0/search?q=search(query: "current weather Mexico City")



{'prompt_tokens': 6165, 'completion_tokens': 240, 'total_tokens': 6405, 'prompt_token_details': {'cached_tokens': 0}}

In [22]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: ☔🌩️ Weather Update for Mexico City! 🌆

Today is an adventure in wetness—bring your umbrella and maybe a rubber duck! 🚶‍♂️💦

- Current Temperature: 21°C (feels like a "not so spicy taco" 🌮)
- High/Low: 22°C / 13°C (Mother Nature's version of "mild sauce") 🌡️
- Humidity: 62% (like walking through a moist tortilla)
- Thunderstorms all day! ⛈️⚡ (Yup, the sky is throwing a drum party and forgot the invite list)
- Rain probability: 100% (try not to float away, amigos!)
- Sunrise/Sunset: 5:57am / 7:17pm (there’s a sun somewhere behind those clouds... probably?)

So, grab your rain boots and dance like no one’s watching—because everyone’s hiding from the thunder anyway! 😆🌂

Stay dry, stay awesome!【11:4†Weather today - Mexico City, Mexico】
URL Citation: [Weather today - Mexico City, Mexico](https://www.weather-atlas.com/en/mexico/mexico-city)


### Post processing

In [ ]:
# List all agents in the project
print("Listing all agents in the project:")
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")

In [ ]:
# recurse through all agents and delete them adding a test to stop if the agent is not found
for agent in project_client.list_agents():
    try:
        print(f"Deleting agent ID: {agent.id}, Name: {agent.name}")
        project_client.delete_agent(agent.id)
    except Exception as e:
        print(f"Error deleting agent ID: {agent.id}, Name: {agent.name}, Error: {e}")

In [ ]:
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")